# SYMFLUENCE Tutorial 03a — Regional Domain Modeling (Iceland)

## Introduction

This tutorial marks a fundamental shift in scale and approach from single-watershed modeling to regional domain modeling. Building on Tutorials 02a-02c where we modeled individual watersheds, we now model an entire region containing multiple independent drainage systems.

Regional domain modeling treats geographic regions—such as countries or provinces—as the modeling domain rather than single watersheds. This approach encompasses multiple independent watersheds including coastal systems that drain directly to the ocean. Using Iceland as our exemplary case study, we demonstrate how SYMFLUENCE handles complex regional hydrology.

Iceland provides an exceptional demonstration with its 103,000 km² area containing diverse drainage systems from glacial watersheds to volcanic terrain. The island setting provides clear boundaries while offering sufficient complexity to illustrate regional modeling principles. The same SYMFLUENCE workflow seamlessly scales from single-watershed to regional modeling while maintaining consistency and scientific rigor.


## Step 1 — Configuration

We generate a regional configuration starting from the template, specifying bounding box coordinates rather than pour points for regional coverage.

In [ ]:
# Import libraries
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import xarray as xr
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from symfluence import SYMFLUENCE
from symfluence.resources import get_config_template


In [ ]:
from symfluence.resources import get_config_template
SYMFLUENCE_CODE_DIR = Path.cwd().resolve()
# Step 1 — Create regional configuration for Iceland
config_template = get_config_template()

# Load base configuration
with open(config_template, 'r') as f:
    config = yaml.safe_load(f)

# Configure for regional Iceland modeling
config['SYMFLUENCE_CODE_DIR'] = str(SYMFLUENCE_CODE_DIR)
#config['SYMFLUENCE_DATA_DIR'] = '/path/to/your/SYMFLUENCE_data'  # ← Update this path

# Regional domain settings
config['DOMAIN_NAME'] = 'Iceland'
config['DOMAIN_DEFINITION_METHOD'] = 'delineate'
config['DELINEATION_METHOD'] = 'stream_threshold'

config['DELINEATE_COASTAL_WATERSHEDS'] = True
config['DELINEATE_BY_POURPOINT'] = False
config['CLEANUP_INTERMEDIATE_FILES'] = False


config['BOUNDING_BOX_COORDS'] = '66.5/-25.0/63.0/-13.0'  # Iceland bounding box
config['POUR_POINT_COORDS'] = '64.01/-16.01'  # random pour point in iceland 
config['STREAM_THRESHOLD'] = 2000  # Higher threshold for regional scale

# Experiment settings
config['EXPERIMENT_ID'] = 'regional_tutorial'
config['EXPERIMENT_TIME_START'] = '2010-01-01 01:00'
config['EXPERIMENT_TIME_END'] = '2010-12-31 23:00'

# Model settings
config['HYDROLOGICAL_MODEL'] = 'SUMMA'
config['ROUTING_MODEL'] = 'mizuRoute'
config['SUB_GRID_DISCRETIZATION'] = 'GRUs'

# Save configuration
config_path = Path("./config_iceland_tutorial.yaml")
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print(f"✅ Regional Iceland configuration saved: {config_path}")

In [ ]:
# Initialize SYMFLUENCE
cf = SYMFLUENCE(config_path, visualize=True)
project_dir = cf.managers['project'].setup_project()

# Create pour point file (required for technical reasons but not used for regional delineation)
pour_point_path = cf.managers['project'].create_pour_point()

print(f"✅ Project structure created at: {project_dir}")

## Step 2 — Regional domain delineation

Regional delineation operates differently from watershed delineation by identifying all independent drainage systems within the bounding box rather than tracing upstream from a single outlet.

### Step 2a — Geospatial attributes

Acquire elevation, land cover, and soil data for the regional domain.

In [ ]:
# Step 2a — Attribute acquisition
#cf.managers['geospatial'].acquire_geospatial_attributes()
print("✅ Regional geospatial attributes acquired")

### Step 2b — Regional discretization

Delineate multiple independent watersheds within the region.

In [ ]:
# Step 2b — Regional delineation
cf.managers['domain'].define_domain()
print("✅ Regional multi-watershed delineation complete")

In [ ]:
# Step 2b — Elevation-based HRU discretization
hru_path = cf.managers['domain'].discretize_domain()
print("✅ Elevation-based HRU discretization complete")

### Step 2c — Verification

Verify the regional domain structure and visualize the multiple drainage systems.

In [ ]:
# Step 2c — Domain verification and visualization (using native SYMFLUENCE plotting)
from IPython.display import Image, display

# Use native visualization for domain overview
plot_path = cf.managers['domain'].visualize_domain()
print(f"Domain plot saved to: {plot_path}")

if plot_path:
    display(Image(filename=str(plot_path)))

# Print domain summary
catchment_path = project_dir / 'shapefiles' / 'catchment'
network_path = project_dir / 'shapefiles' / 'river_network'

if catchment_path.exists() and network_path.exists():
    import geopandas as gpd
    basins_gdf = gpd.read_file(list(catchment_path.glob('*.shp'))[0])
    rivers_gdf = gpd.read_file(list(network_path.glob('*.shp'))[0])
    
    if 'DSLINKNO' in rivers_gdf.columns:
        outlet_count = len(rivers_gdf[rivers_gdf['DSLINKNO'] == -1])
    else:
        outlet_count = 'Unknown'
    
    print(f"\n📊 Regional Domain Summary:")
    print(f"   Watersheds: {len(basins_gdf)}")
    print(f"   Coastal outlets: {outlet_count}")
    print(f"   Stream segments: {len(rivers_gdf)}")
    print(f"   Total area: {basins_gdf.geometry.area.sum() / 1e6:.0f} km²")

## Step 3 — Data acquisition and preprocessing

Acquire and process meteorological forcing and observational data across the regional domain.

### Step 3a — Meteorological forcing

Acquire regional forcing data distributed across multiple watersheds.

In [ ]:
# Step 3a — Forcing acquisition
# cf.managers['data'].acquire_forcings()
print("✅ Regional forcing acquisition complete")

### Step 3b — Model-agnostic preprocessing

Standardize data formats for model consumption across the regional domain.

In [ ]:
# Step 3c — Preprocessing
cf.managers['data'].run_model_agnostic_preprocessing()
print("✅ Regional preprocessing complete")

## Step 4 — Regional model configuration and execution

Configure and run SUMMA-mizuRoute for the regional multi-watershed system.

In [ ]:
# Step 4a — Model configuration
cf.managers['model'].preprocess_models()
print("✅ Regional model configuration complete")

In [ ]:
# Step 4b — Model execution
print(f"Running {config['HYDROLOGICAL_MODEL']} with {config['ROUTING_MODEL']} for regional domain (This may take some time)...")
cf.managers['model'].run_models()
print("✅ Regional simulation complete")

## Step 5 — Regional evaluation

Evaluate model performance across multiple outlets and analyze regional patterns.

In [ ]:
# Step 5 — Regional evaluation
simulation_dir = project_dir / 'simulations' / config['EXPERIMENT_ID']
summa_dir = simulation_dir / 'SUMMA'
routing_dir = simulation_dir / 'mizuRoute'

if summa_dir.exists() and routing_dir.exists():
    # Load simulation outputs
    summa_files = list(summa_dir.glob('*day.nc'))
    routing_files = list(routing_dir.glob('*.nc'))
    
    if summa_files and routing_files:
        summa_data = xr.open_dataset(summa_files[0])
        routing_data = xr.open_dataset(routing_files[0])
        
        print(f"\n📊 Regional Simulation Summary:")
        print(f"   Simulation period: {len(summa_data.time)} days")
        print(f"   HRUs simulated: {len(summa_data.hru)}")
        print(f"   Stream segments: {len(routing_data.seg)}")
        
        # Simple visualization
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Regional SWE
        if 'scalarSWE' in summa_data:
            swe_mean = summa_data['scalarSWE'].mean(dim='time')
            axes[0, 0].hist(swe_mean.values, bins=30, edgecolor='black', alpha=0.7)
            axes[0, 0].set_xlabel('Mean SWE (mm)')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_title('Regional Snow Distribution')
            axes[0, 0].grid(True, alpha=0.3)
        
        # Outlet flow distribution
        if 'IRFroutedRunoff' in routing_data:
            outlets = rivers_gdf[rivers_gdf['DSLINKNO'] == -1]['LINKNO'].values
            outlet_flows = routing_data['IRFroutedRunoff'].sel(seg=outlets).mean(dim='time')
            axes[0, 1].hist(outlet_flows.values, bins=30, edgecolor='black', alpha=0.7, color='blue')
            axes[0, 1].set_xlabel('Mean Flow (m³/s)')
            axes[0, 1].set_ylabel('Number of Outlets')
            axes[0, 1].set_title('Coastal Outlet Flow Distribution')
            axes[0, 1].grid(True, alpha=0.3)
        
        # Time series sample
        if 'IRFroutedRunoff' in routing_data and len(outlets) > 0:
            sample_outlet = outlets[0]
            flow_ts = routing_data['IRFroutedRunoff'].sel(seg=sample_outlet)
            axes[1, 0].plot(flow_ts.time, flow_ts.values, linewidth=1)
            axes[1, 0].set_xlabel('Time')
            axes[1, 0].set_ylabel('Flow (m³/s)')
            axes[1, 0].set_title(f'Sample Outlet (Segment {sample_outlet})')
            axes[1, 0].grid(True, alpha=0.3)
        
        # Regional runoff summary
        if 'scalarTotalRunoff' in summa_data:
            runoff_mean = summa_data['scalarTotalRunoff'].mean(dim='time')
            axes[1, 1].hist(runoff_mean.values, bins=30, edgecolor='black', alpha=0.7, color='green')
            axes[1, 1].set_xlabel('Mean Runoff (mm/day)')
            axes[1, 1].set_ylabel('Frequency')
            axes[1, 1].set_title('Regional Runoff Distribution')
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.suptitle(f'Regional Analysis — {config["DOMAIN_NAME"]}', 
                     fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        summa_data.close()
        routing_data.close()
    else:
        print("⚠️  Simulation outputs not found")
else:
    print("⚠️  Simulation directories not found")

print("\n✅ Regional evaluation complete")